
# Accrued interest and investment history calculator
##### Author: Hathan Khatkar
##### Tracks a users investment into a particular deal, by calculating accrued interest, taking into account partial repayments, rate changes and compound / simple interest, over a monthly basis

#### INSTRUCTIONS:
#####Edit the INPUT DATA cell below to define the investment layout then run the rest of the cells to generate a monthly investment history for a particular deal

In [70]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
import plotly.io as pio
import pandas as pd
import json
from sqlalchemy import create_engine
import calendar
import math
from datetime import datetime, timedelta

## Input data

In [71]:
# INPUT DATA
#-----------------------------------------------------------

invested_amount = 67404.48 # Initial invested amount
flag_compound = 1 #1 if compound/ 0 if simple interest

#Data is aggregated by each change is rate of return, therefore each line indicates the period of which a specific rate of return in active
data = {
    'rate_of_return': [0.005670, 0.006565, 0.007998],       #rate of return is MONTHLY interest rate
    'START': ['2021-07-24', '2023-03-13', '2023-07-19'],    #START is date the current interest rate is applied
    'repayment_date': [np.nan, '2023-06-30', '2024-05-24'], #repayment_date is the date a partial repayment may be made to the investor
    'END': ['2023-03-13', '2023-07-19', None],              # END is the date the interest rate period ends
    'capital_amount': [np.nan, 7750.74, 59653.74]           #capital amount
}

df_combined = pd.DataFrame(data)
display(df_combined)

,rate_of_return,START,repayment_date,END,capital_amount
0,0.005670,2021-07-24,NaN,2023-03-13,NaN
1,0.006565,2023-03-13,2023-06-30,2023-07-19,7750.74
2,0.007998,2023-07-19,2024-05-24,None,59653.74


## General helper functions (for calculating time periods)

In [72]:

def days_in_month(year, month): #gets the number of days in a month
    return calendar.monthrange(year, month)[1]

def months_between(d1, d2):
    return (d1.year - d2.year) * 12 + d1.month - d2.month

def days_between(dt1, dt2):
    delta = dt2 - dt1
    return abs(delta.days)

# to calculate days to the next 1st of the month
def days_to_next_month(d):
    next_month = (d.replace(day=1) + pd.DateOffset(months=1)).replace(day=1)
    return (next_month - d).days

# to calculate days since the last 1st of the month
def days_since_last_month(d):
    last_month = d.replace(day=1)
    return (d - last_month).days

df_combined['START'] = pd.to_datetime(df_combined['START'])
df_combined['END'] = pd.to_datetime(df_combined['END'])
df_combined['repayment_date'] = pd.to_datetime(df_combined['repayment_date'])



## Calculate interest rate and value history for one particular rate of return period

In [73]:
def calculate_history_for_rate(start_date, end_date, rate, val, deduction, curr_rate, edit_rate, isCompound, original_invested):
    orig_start = start_date
    orig_end = end_date
    orig_invested = original_invested

    # calculate the value of an investment after a specified number of days (both compound and simple)
    def calculate_daily_return(start_date, days, rate, isCompound, val, orig_invested):
        num_days = days_in_month(start_date.year, start_date.month)
        if isCompound:
            daily_return = 1 + (rate * (1 / num_days))
            for _ in range(days):
                val *= daily_return
        else:
            daily_return = (rate * (1 / num_days))
            for _ in range(days):
                val += orig_invested * daily_return
        return val

    # calculate the value of an investment after a specified number of months (both compound and simple)
    def calculate_monthly_return(start_date, months, rate, isCompound, val, orig_invested):
        for _ in range(months):
            if isCompound:
                val *= (1 + rate)
            else:
                val += (orig_invested * rate)
            print(f"Month {_+1}/{months}, Value: {val}")
        return val

    # deduction
    def adjust_for_deduction(val, deduction):
        val -= deduction
        return val

    # computes the accrued interest based on whether the interest is compounded or not
    # If the entire period falls within a single month, calculate the new rate using the proportion of the days in that month
    # otherwise calcuate interest for the full months
    def calculate_accrued_interest(orig_start, orig_end, days, days_until_next_month, months, days_since_last_month_val, rate, curr_rate, isCompound):
        if isCompound:
            if orig_start.year == orig_end.year and orig_start.month == orig_end.month:
                new_rate = curr_rate * (1 + rate * (days / days_in_month(orig_start.year, orig_start.month)))
            else:
                new_rate = curr_rate * (1 + rate * (days_until_next_month / days_in_month(orig_start.year, orig_start.month))) * \
                           math.pow(1 + rate, months) * (1 + rate * (days_since_last_month_val / days_in_month(orig_end.year, orig_end.month)))
        else:
            if orig_start.year == orig_end.year and orig_start.month == orig_end.month:
                months_total = days / days_in_month(orig_start.year, orig_start.month)
                new_rate = curr_rate + rate * months_total
            else:
                months_total = days_until_next_month / days_in_month(orig_start.year, orig_start.month) + months + \
                               days_since_last_month_val / days_in_month(orig_end.year, orig_end.month)
                new_rate = curr_rate + rate * months_total
        return new_rate

    # Initialize days
    days = 0

    # When repayment date/end date is in the same month as the start date
    if (orig_start.year == orig_end.year and orig_start.month == orig_end.month):
        days = days_between(start_date, end_date)
        print(start_date)
        print(end_date)
        print(days)
        val = calculate_daily_return(start_date, days, rate, isCompound, val, orig_invested)
        print("Current value: ", val)
        print("\n")
    else:
      #else repayment date/end date is in a different month as the start date

        #1. Calculate partial month value (from Start date to the start of the next month)
        print("Calculating for dates from ", start_date)
        print("To ", end_date)
        if start_date.day != 1:
            days_until_next_month = days_to_next_month(start_date)
            print("Start date: " + str(start_date))
            print("Days until next month: " + str(days_until_next_month))
            val = calculate_daily_return(start_date, days_until_next_month, rate, isCompound, val, orig_invested)
            start_date += timedelta(days=days_until_next_month)
            print("Current value: ", val)
            print("\n")
        else:
            days_until_next_month = 0

        #2. Calculate monthly history (invested value) (from start of following month from start date, to start of the current month of end date)
        months = months_between(end_date, start_date)
        if months == -1:
            months = 0
            print("Within same month, therefore no monthly interest applied")
        else:
            print("Start date (nearest following month):", start_date)
            print("End date (nearest preceding month):", datetime(end_date.year, end_date.month, 1))
            print("Months between:", months)

        val = calculate_monthly_return(start_date, months, rate, isCompound, val, orig_invested)
        print("\n")

        if months == 0:
            print("(No Months) Value: ", val)

        #3. Calculate partial month value (from start of the current month of end date, to the end date)
        if end_date.day != 1:
            days_since_last_month_val = days_since_last_month(end_date)
            print("End date: ", end_date)
            print("Days since last month: ", days_since_last_month_val)
            val = calculate_daily_return(end_date, days_since_last_month_val, rate, isCompound, val, orig_invested)
            print("Current value: ", val)
            print("\n")
        else:
            days_since_last_month_val = 0

    # Apply deducion if there is one for the time period
    val = adjust_for_deduction(val, deduction)
    print(f"Value after capital deduction: {val}")

    # The accrued rate is editted when compound
    print("Accrued over period (Date invested to latest repayment):")
    if edit_rate:
        new_rate = calculate_accrued_interest(orig_start, orig_end, days, days_until_next_month, months, days_since_last_month_val, rate, curr_rate, isCompound)
    else:
        new_rate = curr_rate

    print("Accrued interest rate: ", new_rate)
    print("Accrued interest: ", ((new_rate - 1) * orig_invested))
    print("\n")

    return val, new_rate

## Iterate history for each rate of return's period

In [74]:
INVESTED = invested_amount
current_value = invested_amount
current_rate = 1
print("Initial investment: ", current_value)
print("---------------------------------------\n")

# Iterate for new rate of return period
for index, row in df_combined.iterrows():
    print("-------------" + "RATE " + str(row["rate_of_return"]) + " -------------")

    # If there is no repayment in this period then calculate history from start to end date setting deduction to 0
    if (pd.isna(row['repayment_date'])):

        current_value, current_rate = calculate_history_for_rate(row['START'], row['END'], row['rate_of_return'], current_value, 0, current_rate, True, flag_compound, INVESTED)

    #  Else we need to split the period in two, 1.calculating from start date to repayment date, 2.applying the deduction, 3.then calculate repayment date to end date
    else:
        current_value, current_rate = calculate_history_for_rate(row['START'], row['repayment_date'], row['rate_of_return'], current_value, row['capital_amount'], current_rate, True, flag_compound, INVESTED)
        print("---------------------------------")
        print("Total Capital deduction: " , (row['capital_amount']))
        print("Repayment: " , (row['capital_amount'] * (current_rate -1)))
        print("---------------------------------")
        if (pd.isna(row['END'])):
            print("")
        else:
            current_value, current_rate = calculate_history_for_rate(row['repayment_date'], row['END'], row['rate_of_return'], current_value, 0, current_rate, True, flag_compound, INVESTED)

    print("---------------------------------")
    print("Current value: ", current_value)
    print("---------------------------------\n")


Initial investment:  67404.48
---------------------------------------

-------------RATE 0.00567 -------------
Calculating for dates from  2021-07-24 00:00:00
To  2023-03-13 00:00:00
Start date: 2021-07-24 00:00:00
Days until next month: 8
Current value:  67503.17113551946


Start date (nearest following month): 2021-08-01 00:00:00
End date (nearest preceding month): 2023-03-01 00:00:00
Months between: 19
Month 1/19, Value: 67885.91411585786
Month 2/19, Value: 68270.82724889477
Month 3/19, Value: 68657.92283939601
Month 4/19, Value: 69047.21326189539
Month 5/19, Value: 69438.71096109034
Month 6/19, Value: 69832.42845223974
Month 7/19, Value: 70228.37832156394
Month 8/19, Value: 70626.57322664722
Month 9/19, Value: 71027.02589684231
Month 10/19, Value: 71429.74913367741
Month 11/19, Value: 71834.75581126536
Month 12/19, Value: 72242.05887671524
Month 13/19, Value: 72651.67135054621
Month 14/19, Value: 73063.60632710381
Month 15/19, Value: 73477.8769749785
Month 16/19, Value: 73894.49653